In [2]:
# %pip install -e /Users/mvalatsou/PhD/Repos/BOREAS

from __future__ import annotations
from pathlib import Path
import argparse
import numpy as np

from boreas.config import load_config_toml
from boreas import ModelParams, MassLoss, Fractionation
from boreas.config import apply_params_from_config, build_inputs_from_config, fractionation_runtime_args


In [ ]:
def main(cfg_path: Path):
    # --- load config ---
    with cfg_path.open("rb") as f:
        cfg = tomllib.load(f)

    # --- init ---
    params = ModelParams()
    fx_args = apply_params_from_config(cfg, params)
    mass_loss = MassLoss(params)
    fractionation = Fractionation(params)

    # --- bulk inputs (mass[g], radius[cm], teq[K]) ---
    mass, radius, teq = build_inputs_from_config(cfg, params)

    # --- run ---
    ml_results  = mass_loss.compute_mass_loss_parameters(mass, radius, teq)
    frac_kwargs = fractionation_runtime_args(cfg)
    f_results   = fractionation.execute(ml_results, mass_loss, **frac_kwargs)

    r0 = f_results[0]
    print("\n Done.")
    print("Planet:", fx_args["planet_name"])
    print("Regime:", r0.get("regime"),
          "RXUV[cm]:", r0.get("RXUV"),
          "Mdot[g/s]:", r0.get("Mdot"))
    print("light_major:", r0.get("light_major_i"),
          "heavy_major:", r0.get("heavy_major_j"))
    print("T_outflow[K]:", r0.get("T_outflow"),
          "mu_outflow:", r0.get("mmw_outflow"))

if __name__ == "__main__":
    # default to your K2-18 b example
    default_cfg = Path("../examples/configs/k2-18b.toml")
    main(default_cfg)


In [ ]:
# # pretty print a few fields
# # print a tiny summary
# r0 = f_results[0]
# print("\nDone.")
# print("Regime:", r0.get("regime"), "  RXUV[cm]:", r0.get("RXUV"), "  Mdot[g/s]:", r0.get("Mdot"))
# print("light_major:", r0.get("light_major_i"), "  heavy_major:", r0.get("heavy_major_j"))


# # sol = [
# #     "planet", "Teq", "FXUV", "Mdot", "RXUV", "cs", "mmw_outflow", "T_outflow",
# #     "phi_H_num", "phi_O_num", "phi_C_num", "phi_N_num", "phi_S_num",
# #     "x_O", "x_C", "x_N", "x_S",
# #     "regime"
# # ]